## C9_Wk3_Assignment_RD

## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

# Part1) Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name

In [2]:
# import libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         395 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

In [3]:
# install beautifulSoup for extracting table from websites
!pip install BeautifulSoup4
print ('installed')

     |████████████████████████████████| 122kB 5.8MB/s eta 0:00:01
installed


In [4]:

pip install lxml


     |████████████████████████████████| 5.5MB 3.3MB/s eta 0:00:01███████████▏                | 2.6MB 3.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
# import more libraries
import requests 
from bs4 import BeautifulSoup 

print('Libraries imported.')

Libraries imported.


In [6]:
## import table from website using beautifulSoup

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')


In [7]:
# Convert table to dataframe

#create dataframe columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df_root = pd.DataFrame(columns = column_names)


# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    rows=[]
    for td_cell in tr_cell.find_all('td'):
        rows.append(td_cell.text.strip())
    if len(rows)==3:
        df_root.loc[len(df_root)] = rows
    
     

In [8]:
#display dataframe
df_root.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Data Cleaning

In [9]:
# count 'Not Assigned' in Borough
df_root['Borough'].value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [10]:
# drop rows where Borough is unassigned and combine.
rows_drop = df_root[df_root['Borough'].apply(lambda x: x.startswith('Not'))].index
df_root2 = df_root.drop(rows_drop)
df_root2.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
#count 'Not Assigned' in dataframe ['Borough']
df_root2['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

In [12]:
# Check for Postalcode duplicates
df_root2.groupby('Postalcode')
df_root2.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [13]:
# count 'Not Assigned' in datafame ['Neighborhood']
df_root2['Neighborhood'].value_counts()


Downsview                                                                                                                                 4
Don Mills                                                                                                                                 2
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto                                                      1
Harbourfront East, Union Station, Toronto Islands                                                                                         1
Berczy Park                                                                                                                               1
Upper Rouge                                                                                                                               1
Humewood-Cedarvale                                                                                                                        1
Bayview Village     

In [25]:
# The requested dataframe
df_root2.head(9)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"


In [15]:
#The dataframe shape
df_root2.shape

(103, 3)

### END OF PART1

# PART2) Utilize the Foursquare location data, to get the latitude and the longitude coordinates of each neighborhood.

In [16]:
# import geocoder
!pip install geocoder

     |████████████████████████████████| 102kB 5.9MB/s ta 0:00:011
     |████████████████████████████████| 92kB 6.0MB/s eta 0:00:011
     |████████████████████████████████| 829kB 6.8MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [29]:
# import geocoder
import geocoder 

#Convert Postalcode column of df_root2 to List object postal_code
postal_code = df_root2['Postalcode'].tolist()


In [35]:
## Trying get coordinates using Geocoder Python package

for codes in postal_code:
    geolocator = Nominatim(user_agent="postal_code_explorer")
    location = geolocator.geocode(codes)
    latitude = location.latitude
    longitude = location.longitude
    
    print('The geograpical coordinates of', codes, 'are {}, {}.'.format(latitude, longitude))

## Not all coordinates are obtained.

The geograpical coordinates of M3A are 49.484606, 8.466197496745698.
The geograpical coordinates of M4A are 49.4842897, 8.46700036809857.
The geograpical coordinates of M5A are 45.44058845, 28.018025356938132.
The geograpical coordinates of M6A are 53.7941642, -1.7520058.
The geograpical coordinates of M7A are 44.4281982, 26.165950956173265.


AttributeError: 'NoneType' object has no attribute 'latitude'

In [49]:
## Trying get coordinates using Geocoder Python package and Loops

latitide = None
longitude = None


for codes in postal_code:
    while (latitude == None or longitude == None):
        geolocator = Nominatim(user_agent="postal_code_explorer")
        location = geolocator.geocode(codes)
        latitude = location.latitude
        longitude = location.longitude

    print('The geograpical coordinates of', codes, 'are {}, {}.'.format(latitude, longitude))

# Cannot get it to work!

In [60]:
## Trying get coordinates using http://cocl.us/Geospatial_data
# Download Geospatial Coordinate File

!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [63]:
## Trying get coordinates using http://cocl.us/Geospatial_data
# Convert csv to pandas dataframe
df_coord = pd.read_csv("Geospatial_Coordinates.csv")
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [68]:
# Columns in df_root2
df_root2.columns


Index(['Postalcode', 'Borough', 'Neighborhood'], dtype='object')

In [67]:
#Columns in df_coord
df_coord.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [72]:
# Rename Column 'Postal Code' to 'Postalcode' in df_coord
df_coord.rename(columns = {'Postal Code':'Postalcode'}, inplace = True) 


In [76]:
# Full outer join of  df_root2 and df_coord usng column 'Postalcode'
df_merged = pd.merge(left=df_root2, right=df_coord, how='outer', left_on='Postalcode', right_on='Postalcode')
df_merged.head()


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### End Of Part 2.